In [ ]:
# Do not use it if you have already installed the requirements file
# !pip install langchain langchain_openai langchain-community langchain_experimental gradio psycopg2 python-dotenv ag2
# !pip uninstall pydantic
# !pip install pydantic==2.9.2
# !pip install autogen-agentchat~=0.2

In [ ]:
import autogen
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
import gradio as gr
import asyncio
import psycopg2
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from langchain.cache import InMemoryCache
from langchain_experimental.sql.base import SQLDatabaseChain

In [2]:
# Load environment variables from the .env file from the same directory as notebook 
load_dotenv()

# Retrieve environment variables
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_DB = os.getenv('POSTGRES_DB')
AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')

In [ ]:
# Construct the database URI
shipment_db_uri = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
crm_db_uri = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Establish database connections
shipment_db = SQLDatabase.from_uri(shipment_db_uri)
crm_db = SQLDatabase.from_uri(crm_db_uri)

In [4]:
# Initialize the Azure OpenAI language model
azure_llm = AzureChatOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
  api_key=AZURE_OPENAI_KEY,
  api_version="2024-10-21",
  deployment_name=AZURE_OPENAI_DEPLOYMENT,
)

In [5]:
# Query functions for each database
def query_shipment(query):
    return shipment_chain.invoke(query)

def query_crm(query):
    return crm_chain.invoke(query)

# Function to retrieve database schema information
def get_schema_info():
    from sqlalchemy import text
    import json  # Import json module to convert data to JSON string
    with shipment_db._engine.connect() as connection:
        query = text("""
        SELECT
            cols.table_schema,
            cols.table_name,
            cols.column_name,
            cols.data_type,
            cols.is_nullable,
            cons.constraint_type,
            cons.constraint_name,
            fk.references_table AS referenced_table,
            fk.references_column AS referenced_column
        FROM information_schema.columns cols
        LEFT JOIN information_schema.key_column_usage kcu
            ON cols.table_schema = kcu.table_schema
            AND cols.table_name = kcu.table_name
            AND cols.column_name = kcu.column_name
        LEFT JOIN information_schema.table_constraints cons
            ON kcu.table_schema = cons.table_schema
            AND kcu.table_name = cons.table_name
            AND kcu.constraint_name = cons.constraint_name
        LEFT JOIN (
            SELECT
                rc.constraint_name,
                kcu.table_name AS references_table,
                kcu.column_name AS references_column
            FROM information_schema.referential_constraints rc
            JOIN information_schema.key_column_usage kcu
                ON rc.unique_constraint_name = kcu.constraint_name
        ) fk
            ON cons.constraint_name = fk.constraint_name
        WHERE cols.table_schema = 'public'
        ORDER BY cols.table_schema, cols.table_name, cols.ordinal_position;
        """)
        result = connection.execute(query)
        columns = result.keys()
        rows = result.fetchall()
        # Convert the result to a list of dictionaries
        schema_info = [dict(zip(columns, row)) for row in rows]
    return json.dumps(schema_info, indent=2)

# Function to share schema information between agents
def get_shared_schema_info():
    if schema_agent.schema_info is None:
        schema_agent.retrieve_and_store_schema()
    return schema_agent.schema_info

# Method to retrieve and store schema information
def retrieve_and_store_schema(agent):
    schema_info = get_schema_info()
    agent.schema_info = schema_info
    return "Schema information retrieved and stored."

# Method to add a new customer to the CRM database
def add_customer(procedure_name, parameters):
    from sqlalchemy import text
    with crm_db._engine.connect() as connection:
        trans = connection.begin()  # Begin a transaction
        try:
            # Prepare the parameter placeholders
            param_placeholders = ', '.join([f":{k}" for k in parameters.keys()])
            # Construct the SQL command to execute the stored procedure
            sql_command = text(f"CALL {procedure_name}({param_placeholders})")
            # Pass parameters as a dictionary
            result = connection.execute(sql_command, parameters)
            # Commit the transaction
            trans.commit()
            # Return a success message
            return "Customer added successfully."
        except Exception as e:
            trans.rollback()
            return f"An error occurred while executing the stored procedure: {e}"

# Method to create a new shipment in the shipment database
def send_shipment(procedure_name, parameters):
    from sqlalchemy import text
    import json
    with shipment_db._engine.connect() as connection:
        trans = connection.begin()  # Begin a transaction
        try:
            # If 'items' is a list, convert it to JSON string
            if isinstance(parameters.get('items'), list):
                parameters['items'] = json.dumps(parameters['items'])
            # Prepare the parameter placeholders
            param_placeholders = ', '.join([f":{k}" for k in parameters.keys()])
            # Construct the SQL command
            sql_command = text(f"CALL {procedure_name}({param_placeholders})")
            # Execute the stored procedure
            result = connection.execute(sql_command, parameters)
            # Commit the transaction
            trans.commit()
            return "Shipment sent successfully."
        except Exception as e:
            trans.rollback()
            return f"An error occurred while executing the stored procedure: {e}"

In [ ]:
# Language model configuration with functions
llm_config = {
    "config_list": [
        {
        "model": AZURE_OPENAI_DEPLOYMENT,
        "temperature": 0.7,
        "api_key": AZURE_OPENAI_KEY,
        "azure_endpoint": AZURE_OPENAI_ENDPOINT,
        "api_type": "azure",
        "api_version": "2024-10-21"
        }
    ],
    "seed": 42,
    "functions": [
        {
            "name": "query_shipment",
            "description": "Queries the Shipment database based on the provided query",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The SQL query to execute on the shipment database"}
                },
                "required": ["query"]
            }
        },
        {
            "name": "query_crm",
            "description": "Queries the CRM database based on the provided query",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The SQL query to execute on the CRM database"}
                },
                "required": ["query"]
            }
        },
        {
            "name": "get_schema_info",
            "description": "Retrieves the database schema and referential integrity information. Only use 'get_schema_info' to retrieve schema information and store it. Do not do anything else",
            "parameters": {"type": "object", "properties": {}, "required": []}
        },
        {
            "name": "get_shared_schema_info",
            "description": "Provides the stored schema information to other agents.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        },
        {
            "name": "add_customer",
            "description": "Adds a customer to the CRM database by executing the 'add_customer' stored procedure with the provided parameters.",
            "parameters": {
                "type": "object",
                "properties": {
                    "procedure_name": {
                        "type": "string",
                        "description": "The name of the stored procedure to execute (should be 'add_customer')"
                    },
                    "parameters": {
                        "type": "object",
                        "description": "A dictionary of parameters to pass to the stored procedure, including 'name', 'email', 'phone', and 'address'.",
                        "properties": {
                            "name": {"type": "string"},
                            "email": {"type": "string"},
                            "phone": {"type": "string"},
                            "address": {"type": "string"}
                        },
                        "required": ["name", "email", "phone", "address"]
                    }
                },
                "required": ["procedure_name", "parameters"]
            }
        },
        {
            "name": "send_shipment",
            "description": "Sends a shipment by executing the 'send_shipment' stored procedure with the provided parameters.",
            "parameters": {
                "type": "object",
                "properties": {
                    "procedure_name": {
                        "type": "string",
                        "description": "The name of the stored procedure to execute (should be 'send_shipment')"
                    },
                    "parameters": {
                        "type": "object",
                        "description": "Parameters for the stored procedure, including 'customer_id', 'origin_id', 'destination_id', 'shipment_date', 'items', 'status', 'tracking_status', and 'location_id'.",
                        "properties": {
                            "customer_id": {"type": "integer"},
                            "origin_id": {"type": "integer"},
                            "destination_id": {"type": "integer"},
                            "shipment_date": {"type": "string", "format": "date"},
                            "items": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "product_id": {"type": "integer"},
                                        "quantity": {"type": "integer"}
                                    },
                                    "required": ["product_id", "quantity"]
                                }
                            },
                            "status": {"type": "string"},
                            "tracking_status": {"type": "string"},
                            "location_id": {"type": "integer"}
                        },
                        "required": ["customer_id", "origin_id", "destination_id", "shipment_date", "items", "status", "tracking_status", "location_id"]
                    }
                },
                "required": ["procedure_name", "parameters"]
            }
        }
    ]
}


# Initialize the database chains
shipment_chain = SQLDatabaseChain(llm=azure_llm, database=shipment_db, verbose=True)
crm_chain = SQLDatabaseChain(llm=azure_llm, database=crm_db, verbose=True)

# Create assistant agents
shipment_agent = autogen.ConversableAgent(
    name="ShipmentAgent",
    llm_config=llm_config,
    description="Manage shipments in the main database.",
    system_message=(
        "Your role is to query the main database using 'query_shipment'. "
        "Focus on the shipments tables and ensure that all shipments are tracked correctly. You can make SELECT using PostgreSQL queries. Use the 'add_customer' function to call the appropriate stored procedure for adding new customers."
        "For Insert, Update, and Delete operations, have human to validate the operation before making it."
        "Here is an example on how to call add_customer stored procedure: CALL add_customer('marcre@contoso.com', 'marcre@contoso.com', '+1 123 456 7890','1 Main Street, Redmond');"
        "Use 'get_shared_schema_info' from SchemaAgent to retrieve schema information."
    ),
)

crm_agent = autogen.ConversableAgent(
    name="CRMAgent",
    llm_config=llm_config,
    description="Manages customer and product information in the second database.",
    system_message=(
        "Your role is to query the second database using 'query_crm'. "
        "Focus on maintaining the customers and product tables. You can make SELECT using PostgreSQL queries. Use the 'send_shipment' function to call the appropriate stored procedure for creating shipments."
        "For Insert, Update, and Delete operations, have human to validate the operation before making it."
        "Here is an example on how to call 'send_shipment' stored procedure:"
        "CALL send_shipment("
        "   customer_id     := 1,"
        "    origin_id       := 3,"
        "    destination_id  := 2,"
        "    shipment_date   := '2023-10-01',"
        "    items           := '["
        "                            {'product_id': 1, 'quantity': 5},"
        "                            {'product_id': 2, 'quantity': 3}"
        "                        ]'::jsonb,"
        "    status          := 'In Transit',"
        "    tracking_status := 'Departed Origin',"
        "    location_id     := 3"
        ");"
        "Use 'get_shared_schema_info' from SchemaAgent to retrieve schema information. "
    ),
)

schema_agent = autogen.ConversableAgent(
    name="SchemaAgent",
    llm_config=llm_config,
    description="Understands and shares database schema information.",
    system_message=(
        "Your role is to retrieve and understand the database schema and referential integrity constraints."
        "Only use 'get_schema_info' to retrieve schema information and store it. Do not do anything else. And always provide schema information when you start first."
    ),
)

# Register functions with the agents
shipment_agent.register_function(function_map={
    "query_shipment": query_shipment,
    "send_shipment": send_shipment
})
crm_agent.register_function(function_map={"query_crm": query_crm,
    "add_customer": add_customer})
schema_agent.register_function(
    function_map={
        "get_schema_info": get_schema_info,
        "get_shared_schema_info": get_shared_schema_info,
    }
)

# Add schema_info attribute and bind method to schema_agent
import types
schema_agent.schema_info = None
schema_agent.retrieve_and_store_schema = types.MethodType(retrieve_and_store_schema, schema_agent)

In [7]:
# Create a user proxy agent
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 4,
        "work_dir": "groupchat",
        "use_docker": False,
    },
    human_input_mode="ALWAYS", #Using this mode to give input to agents
)

# Set up the group chat and manager
groupchat = autogen.GroupChat(
    agents=[user_proxy, schema_agent, shipment_agent, crm_agent],
    messages=[],
    max_round=30 # Maximum number of rounds in the conversation
)

manager = autogen.GroupChatManager(groupchat=groupchat)

In [ ]:
# Chat with your data use case

user_proxy.initiate_chat(manager, message="Which products with names are currently tracking in transit?") 

In [ ]:
# See messages in the groupchat
print("Current messages in the groupchat:")
for idx, msg in enumerate(manager.groupchat.messages):
    print(f"Message {idx}: {msg}")

In [ ]:
# Chat with your data use case

user_proxy.initiate_chat(
    manager,
    message=(
        "Is Alice Johnson a Customer?"
    )
)

In [ ]:
# Database Development use case

user_proxy.initiate_chat(manager, message="I need to create a Stored Procedure to send shipments. It spans across shipments, shipment_items and shipment_tracking? Shipment_items might have multiple items and can vary. What stored procedure would you propose?") 

In [ ]:

user_proxy.initiate_chat(
    manager,
    message=(
        "I would like to create a stored orodcedure to delete customers. What would be the best way to do this?"
    )
)

In [ ]:
# Act on Data use case: add a customer name

user_proxy.initiate_chat(
    manager,
    message=(
        "Can you add Marc with email address marcr@contoso.com, phone number +1 123 456 7890 and address in 1 Main Street, Redmond?"
    )
)

In [ ]:
# Act on Data use case: add a customer name with incomplete information

user_proxy.initiate_chat(
    manager,
    message=(
        "Can you add Marc with email address marcre@contoso.com, phone number +1 123 456 7890?"
    )
)

In [ ]:
# Act on Data use case: Create a shipment using stored procedure send_shipment writing across 3 tables

user_proxy.initiate_chat(
    manager,
    message=(
        "Can you create a new shipment of 1 Laptop and 1 Smartphone to Cathy Lee and ensure shipment is updated to Departed Origin from the location in New York and towards Los Angeles date is today? Ask questions if you have doubts."
    )
)

In [ ]:
# Act on Data use case: Create a shipment using stored procedure send_shipment writing across 3 tables

user_proxy.initiate_chat(
    manager,
    message=(
        "Can you create a new shipment of 1 Laptop and 1 Smartphone to Cathy Lee and ensure shipment is updated to Departed Origin towards Los Angeles date is today? Ask questions if you miss information."
    )
)

In [10]:
import asyncio

# Initialize chat history
chat_history = []

async def process_user_input(user_message, chat_history):
    # Append your message with role 'assistant' (to appear on the left)
    chat_history.append({"role": "assistant", "content": user_message})

    # Append a placeholder for the agent's response with role 'user' (to appear on the right)
    placeholder_index = len(chat_history)
    chat_history.append({"role": "user", "content": "Processing..."})

    # Return the updated chat history immediately
    yield chat_history, chat_history

    # Now process the agent's response
    # Use the user_proxy agent to process the message
    await asyncio.to_thread(user_proxy.initiate_chat, manager, message=user_message)

    # Collect messages from the agents
    agent_messages = [
        msg for msg in manager.groupchat.messages if msg.get("role", "") != "System"
    ]

    # Remove the placeholder
    if placeholder_index < len(chat_history):
        chat_history.pop(placeholder_index)

    # Append each agent's message to the chat history
    for msg in agent_messages:
        name = msg.get("name", "Agent")
        content = msg.get("content", "")
        role = "user"  # Agents' messages will appear on the right

        # Include the agent's name in the content
        content_with_name = f"**{name}**: {content}"

        # Append to chat history
        chat_history.append({"role": role, "content": content_with_name})

        # Yield after each agent's message to update the UI
        yield chat_history, chat_history

    # Return the final chat history
    yield chat_history, chat_history

def gradio_chat_interface():
    with gr.Blocks() as demo:
        chat_history_state = gr.State([])

        gr.Markdown("# Multi-Agent Chat Interface")

        with gr.Row():
            chatbot = gr.Chatbot(type="messages")  # Use 'messages' format
        with gr.Row():
            user_input = gr.Textbox(
                placeholder="Type your message here...",
                show_label=False
            )
            send_button = gr.Button("Send")
            clear_button = gr.Button("Clear Chat")

        async def on_user_message(user_message, chat_history):
            if user_message:
                # Process the user input and get the updated chat history
                # Use a generator to handle incremental updates
                response = process_user_input(user_message, chat_history)
                async for chat_history_update, chat_history_state_update in response:
                    await asyncio.sleep(0)
                    yield gr.update(value=chat_history_update), chat_history_state_update

        send_button.click(
            on_user_message,
            inputs=[user_input, chat_history_state],
            outputs=[chatbot, chat_history_state]
        )

        user_input.submit(
            on_user_message,
            inputs=[user_input, chat_history_state],
            outputs=[chatbot, chat_history_state]
        )

        clear_button.click(
            lambda: ([], []),
            None,
            [chatbot, chat_history_state],
            queue=False
        )

    return demo

In [ ]:
# Run the Gradio interface
demo = gradio_chat_interface()
demo.launch()